In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install fsspec

In [3]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


<h2 style="color:red;">Installing Intel Packages</h2>

In [5]:
pip install scikit-learn-intelex


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install intel-tensorflow

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 MB 1.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 1.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0


In [6]:
pip install onednn-cpu-gomp

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install onednn


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.3/253.3 MB 1.5 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 3.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


<h3 style="color:blue;">Training the model in Intel Dev Cloud Using Intel TensorFlow Packages</h3>

In [2]:

#0-offensive 1-Hate 2-Neither

import numpy as np
import pandas as pd
import pickle
#-------------------------------------using intel packages------------------------------------------
#import onednn as dnn
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#-----------------------------------------------------------------------------------------------------

data=pd.read_csv('data/labeled_data.csv')
#print(data)
data = data.dropna()
#X = ['hi', 'idiot', 'u are black']
#y = [2, 0, 1]
X=data['tweet']
y=data['class']
#print('X',X)
#print(y)
def create_and_train_model(X, y):
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    X_padded = pad_sequences(sequences)

    
    X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

    
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_padded.shape[1]))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=len(np.unique(y_encoded)), activation='softmax'))  

    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

    
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")

    model.save('D:/content_moderation_model.h5')
    print("Model saved.")

    
    with open('D:/tokenizer.pkl', 'wb') as file:
        pickle.dump(tokenizer, file)

    with open('D:/ label_encoder.pkl', 'wb') as file:
        pickle.dump(label_encoder, file)

    print("Tokenizer and LabelEncoder saved.")

    

    return model, tokenizer, label_encoder

create_and_train_model(X,y)



2024-03-09 10:00:07.553163: I tensorflow/core/platform/cpu_feature_guard.cc:181] Beginning TensorFlow 2.15, this package will be updated to install stock TensorFlow 2.15 alongside Intel's TensorFlow CPU extension plugin, which provides all the optimizations available in the package and more. If a compatible version of stock TensorFlow is present, only the extension will get installed. No changes to code or installation setup is needed as a result of this change.
More information on Intel's optimizations for TensorFlow, delivered as TensorFlow extension plugin can be viewed at https://github.com/intel/intel-extension-for-tensorflow.
2024-03-09 10:00:07.553252: I tensorflow/core/platform/cpu_feature_guard.cc:192] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 10:00:22.981649: I tensorflow

Epoch 1/5
778/778 [==============================] - 121s 151ms/step - loss: 0.3781 - accuracy: 0.8758 - val_loss: 0.2461 - val_accuracy: 0.9190
Epoch 2/5
778/778 [==============================] - 117s 151ms/step - loss: 0.1736 - accuracy: 0.9454 - val_loss: 0.2241 - val_accuracy: 0.9314
Epoch 3/5
778/778 [==============================] - 118s 152ms/step - loss: 0.0843 - accuracy: 0.9738 - val_loss: 0.2761 - val_accuracy: 0.9308
Epoch 4/5
778/778 [==============================] - 117s 151ms/step - loss: 0.0458 - accuracy: 0.9864 - val_loss: 0.3104 - val_accuracy: 0.9364
Epoch 5/5
244/244 [==============================] - 8s 31ms/step - loss: 0.3506 - accuracy: 0.9184
Test Accuracy: 0.9183699488639832


/home/u215409/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved.
Tokenizer and LabelEncoder saved.


(<keras.src.engine.sequential.Sequential at 0x7ff237ce2c40>,
 LabelEncoder())

<h2 style="color:blue;">Predicting With the Model Saved in Intel Cloud</h2>

In [9]:
import warnings
warnings.filterwarnings('ignore' )
def predict_with_model(model, tokenizer, label_encoder, input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(input_sequence, maxlen=model.input_shape[1])

   
    predicted_probabilities = model.predict(padded_sequence)
    predicted_class = np.argmax(predicted_probabilities)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    confidence = predicted_probabilities[0][predicted_class]

    if confidence <= 0.6:
        predicted_label=2
    


    
    #print("\nInput Text:")
    #print(f"- {input_text}")

    #print("\nPredicted Probabilities:")
    #print(predicted_probabilities)

    #print("\nPredicted Class:")
    #print(predicted_label)
    return predicted_label

def load_saved_model():
    
    loaded_model = load_model('D:/content_moderation_model.h5')

    
    with open('D:/tokenizer.pkl', 'rb') as file:
        loaded_tokenizer = pickle.load(file)

    with open('D:/ label_encoder.pkl', 'rb') as file:  # Corrected file name
        loaded_label_encoder = pickle.load(file)

    print("Model, Tokenizer, and LabelEncoder loaded.")
    return loaded_model, loaded_tokenizer, loaded_label_encoder


data=pd.read_csv('data/labeled_data.csv')
#print(data)
#X = ['hi', 'idiot', 'u are black']
#y = [2, 0, 1]
X=data['tweet']
y=data['class']
#print('X',X)
#print(y)
def input_model(content):
    #print(content)
    prediction=[]
    train_model=False
    if train_model:
        trained_model, tokenizer, label_encoder = create_and_train_model(X, y)
    else:
        trained_model, tokenizer, label_encoder = load_saved_model()

    result_string = content.replace('"', '').replace("'", '')

    soup = BeautifulSoup(content, 'html.parser')
    p_tag = soup.find_all('p')
    #print("p_tag",p_tag)
    for i in p_tag:
        #print(i)
        cleaned_text = i.get_text(separator=' ', strip=True) if i else ''
        #print(cleaned_text)
        a=predict_with_model(trained_model, tokenizer, label_encoder, cleaned_text)
        prediction.append(a)
        #print("Predictions",prediction)
    
    return prediction

a=input_model('<p><img alt="" src="/media/uploads/2023/12/04/test2-94.jpg" style="height:200px; width:150px" />You are nothing but a waste of space in this bloody world... idiot i hate u so much you black people dont belong here hell dammit shit bloody losers</p>')
for i in a:
    if(i==0 or i==1):
        print("Inappropriate Content")
    else:
        print("Accepted Content")

Model, Tokenizer, and LabelEncoder loaded.
1/1 [==============================] - 1s 517ms/step
Inappropriate Content
